In [5]:
import pandas as pd
import json
import ast
import datetime
import random
from dateutil.relativedelta import relativedelta

id_category = "bdab0052-6bd4-45bd-a98c-f9579b45d2d2"
folder_data = "data/earring"

In [8]:
import pandas as pd
import datetime
import csv
import uuid

# Load data
df = pd.read_csv(f'{folder_data}/tiki_products.csv')

# Convert images to list
def parse_images(img_str):
    if not isinstance(img_str, str):
        return []
    return [u.strip() for u in img_str.split(',')]

df['image_list'] = df['images'].apply(parse_images)
df['avatar'] = df['image_list'].apply(lambda x: x[0] if x else None)

# Convert to Postgres array format
def to_pg_array(url_list):
    if not url_list:
        return "{}"
    return "{" + ",".join(f'"{u}"' for u in url_list) + "}"

df['url_img'] = df['image_list'].apply(to_pg_array)

# Fixed category
df['id_category'] = id_category

# Pricing
df['immediate_purchase_price'] = df['price']
df['starting_price'] = df['price'].apply(lambda p: int(round(p * 0.5, -3)))
df['pricing_step'] = df['price'].apply(lambda p: int(round(max(1000, p * 0.02), -3)))

# Timestamp
now = datetime.datetime.now()
df['posted_date_time'] = now.isoformat()

def generate_end_date():
    now = datetime.datetime.now()
    extra_months = random.randint(1, 3)
    result = now + relativedelta(days=14) + relativedelta(months=extra_months)
    return result.isoformat()

df['end_date_time'] = df.apply(lambda _: generate_end_date(), axis=1)

# User
df['created_by'] = "158f7354-ce82-42ca-a4f2-e276f2fdfb01"
df['updated_by'] = "158f7354-ce82-42ca-a4f2-e276f2fdfb01"

# Rename HTML desc
df.rename(columns={'desc_html': 'description'}, inplace=True)

# Generate product IDs
df['id_product'] = [str(uuid.uuid4()) for _ in range(len(df))]

# Final product columns
final_cols = [
    'id_product',
    'id_category',
    'avatar',
    'name',
    'price',
    'immediate_purchase_price',
    'posted_date_time',
    'end_date_time',
    'description',
    'pricing_step',
    'starting_price',
    'url_img',
    'updated_by',
    'created_by'
]

products_df = df[final_cols]

# Save CSV
products_df.to_csv(f'{folder_data}/normalized_products.csv', index=False)

# ================================
# GENERATE SQL FOR PRODUCT TABLE
# ================================
def escape_sql(val):
    if pd.isna(val):
        return "NULL"
    if isinstance(val, str):
        return "'" + val.replace("'", "''") + "'"
    return str(val)

sql_product = ["-- INSERT INTO product"]

for _, row in products_df.iterrows():
    vals = []
    for col in final_cols:
        if col == "url_img":
            vals.append(f"'{row[col]}'")
        elif isinstance(row[col], str):
            vals.append(escape_sql(row[col]))
        else:
            vals.append(str(row[col]))

    sql_product.append(
        f"INSERT INTO product ({', '.join(final_cols)}) VALUES ({', '.join(vals)});"
    )

with open(f"{folder_data}/insert_product.sql", "w", encoding="utf-8") as f:
    f.write("\n".join(sql_product))


# ==========================================
# GENERATE SQL FOR description_history TABLE
# ==========================================
history_sql = ["-- INSERT INTO description_history"]

for _, row in products_df.iterrows():
    history_sql.append(
        f"INSERT INTO description_history (id_product, time, description) "
        f"VALUES ('{row['id_product']}', '{row['posted_date_time']}', {escape_sql(row['description'])});"
    )

with open(f"{folder_data}/insert_description_history.sql", "w", encoding="utf-8") as f:
    f.write("\n".join(history_sql))

print("Generated normalized_products.csv, insert_product.sql, insert_description_history.sql successfully!")

Generated normalized_products.csv, insert_product.sql, insert_description_history.sql successfully!


In [2]:
pip install psycopg2-binary python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 14.6 MB/s eta 0:00:00
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 0/2 [python-dotenv]
   -------------------- ------------------- 1/2 [psycopg2-binary]
   -------------------- ------------------- 1/2 [psycopg2-binary]
   ---------------------------------------- 2/2 [psycopg2-binary]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import random
import uuid
import psycopg2
from dotenv import load_dotenv
import os

# ================================
# LOAD ENV
# ================================
load_dotenv('../server/.env')

# ================================
# CONNECT DATABASE
# ================================
conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    port=5432,
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database='postgres',
    sslmode='require'
)
cursor = conn.cursor()

# ================================
# FETCH ENDED PRODUCTS WITH SELLER INFO
# ================================
cursor.execute("""
    SELECT p.id_product, p.updated_by as seller_id
    FROM product p
    WHERE p.end_date_time < NOW()
    AND p.status = 'ended_success'
""")
products = [(row[0], row[1]) for row in cursor.fetchall()]
print(f"Loaded {len(products)} ended products with sellers")

# ================================
# FETCH BIDDERS (NON-ADMIN, NON-SELLER ROLE)
# ================================
cursor.execute("""
    SELECT id_user
    FROM public.user
    WHERE role = 'bidder'
""")
bidder_ids = [row[0] for row in cursor.fetchall()]
print(f"Loaded {len(bidder_ids)} bidders")

# ================================
# FETCH SELLERS
# ================================
cursor.execute("""
    SELECT id_user
    FROM public.user
    WHERE role = 'seller'
""")
seller_ids = [row[0] for row in cursor.fetchall()]
print(f"Loaded {len(seller_ids)} sellers")

# ================================
# EXISTING RATING COMBINATIONS
# (id_product, reviewer_id, reviewee_id)
# ================================
cursor.execute("""
    SELECT id_product, reviewer_id, reviewee_id
    FROM rating
""")
existing_combinations = set((row[0], row[1], row[2]) for row in cursor.fetchall())
print(f"Found {len(existing_combinations)} existing ratings")

# ================================
# REVIEW CONTENT
# ================================
positive_reviews_seller_to_bidder = [
    "Người mua thanh toán nhanh chóng, giao dịch thuận lợi!",
    "Buyer rất nhiệt tình và thân thiện.",
    "Giao dịch suôn sẻ, người mua đáng tin cậy.",
    "Người mua thanh toán đúng hẹn, rất hài lòng!",
    "Giao dịch tuyệt vời, buyer rất chuyên nghiệp.",
]

positive_reviews_bidder_to_seller = [
    "Sản phẩm rất đẹp, đúng như mô tả. Giao hàng nhanh!",
    "Người bán nhiệt tình, đóng gói cẩn thận. Rất hài lòng!",
    "Chất lượng tốt, giá cả hợp lý. Sẽ ủng hộ lần sau.",
    "Hàng đẹp, giao nhanh. Cảm ơn shop!",
    "Sản phẩm chính hãng, rất ưng ý.",
    "Người bán thân thiện, hỗ trợ nhiệt tình.",
    "Đóng gói kỹ, hàng về nguyên vẹn.",
    "Sản phẩm đúng mô tả, chất lượng tốt.",
]

negative_reviews_seller_to_bidder = [
    "Người mua thanh toán chậm.",
    "Giao dịch không thuận lợi.",
    "Buyer không phản hồi tin nhắn.",
]

negative_reviews_bidder_to_seller = [
    "Hàng không đúng mô tả.",
    "Giao hàng chậm, đóng gói sơ sài.",
    "Sản phẩm kém chất lượng.",
    "Người bán không nhiệt tình.",
    "Không hài lòng với giao dịch.",
]

# ================================
# GENERATE FAKE RATING DATA
# Simulate both directions:
# - seller rates bidder (reviewer_role = 'seller')
# - bidder rates seller (reviewer_role = 'bidder')
# ================================
rating_data = []
used_combinations = set()

MAX_REVIEWS = 200
attempts = 0

while len(rating_data) < MAX_REVIEWS and attempts < MAX_REVIEWS * 10:
    attempts += 1

    if not products:
        print("No products available")
        break

    # Pick a random product with its seller
    id_product, seller_id = random.choice(products)
    
    # Pick a random bidder (as potential winner)
    if not bidder_ids:
        print("No bidders available")
        break
    bidder_id = random.choice(bidder_ids)
    
    # Make sure bidder is not the seller
    if bidder_id == seller_id:
        continue

    # Randomly decide direction: seller->bidder or bidder->seller
    is_seller_rating = random.choice([True, False])
    
    if is_seller_rating:
        reviewer_id = seller_id
        reviewee_id = bidder_id
        reviewer_role = 'seller'
        positive_reviews = positive_reviews_seller_to_bidder
        negative_reviews = negative_reviews_seller_to_bidder
    else:
        reviewer_id = bidder_id
        reviewee_id = seller_id
        reviewer_role = 'bidder'
        positive_reviews = positive_reviews_bidder_to_seller
        negative_reviews = negative_reviews_bidder_to_seller

    # Check for uniqueness (id_product, reviewer_id, reviewee_id)
    combo = (id_product, reviewer_id, reviewee_id)
    if combo in existing_combinations or combo in used_combinations:
        continue

    used_combinations.add(combo)

    # 80% positive, 20% negative
    if random.random() < 0.8:
        rating_point = 1
        content = random.choice(positive_reviews)
    else:
        rating_point = -1
        content = random.choice(negative_reviews)

    rating_data.append((
        str(uuid.uuid4()),  # id_rating
        id_product,
        reviewer_id,
        reviewee_id,
        reviewer_role,
        rating_point,
        content
    ))

print(f"Generated {len(rating_data)} new ratings")

# ================================
# INSERT INTO DATABASE
# ================================
if rating_data:
    insert_sql = """
        INSERT INTO rating (id_rating, id_product, reviewer_id, reviewee_id, reviewer_role, rating_point, content, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s, NOW())
    """
    cursor.executemany(insert_sql, rating_data)
    conn.commit()
    print("✅ Inserted into rating table successfully")
else:
    print("⚠️ No new ratings to insert")

# ================================
# CLEANUP
# ================================
cursor.close()
conn.close()

Loaded 5 ended products with sellers
Loaded 104 bidders
Loaded 1 sellers
Found 0 existing ratings
Generated 200 new ratings
✅ Inserted into rating table successfully
